In [136]:
import numpy as np
import sys
sys.path.append("../src")
from bert_embedding import *

# Data

## 1. SEC filings

In [4]:
test_link ="https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/test/FIN3"
train_link = "https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt"

In [19]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/test/FIN3.txt -P ../data/sec_ner

In [20]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt -P ../data/sec_ner

## 2. BTC

In [21]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/BTC/CONLL-format/data/h.conll -P ../data/ner_btc

## 3. Wikigold

In [23]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/wikigold/CONLL-format/data/wikigold.conll.txt -P ../data/ner_wikigold

# Data Clean

In [108]:
# load data
def load_data(path, separator = " "):
    with open(path) as f:
        text = f.read().split("\n\n")

    output = []
    for line in text:
        feature_label = []
        line = line.split("\n")
        for entry in line:
            feature_label.append(tuple(entry.split(separator)))
        output.append(feature_label)
    return output

In [109]:
sec_path = "../data/ner_sec/FIN5.txt"
sec = load_data(sec_path)

In [120]:
print(len(sec))
sec[1]

1170


[('This', 'DT', '-', 'O'),
 ('LOAN', 'NN', '-', 'O'),
 ('AND', 'CC', '-', 'O'),
 ('SECURITY', 'NNP', '-', 'O'),
 ('AGREEMENT', 'NNP', '-', 'O'),
 ('dated', 'VBD', '-', 'O'),
 ('January', 'NNP', '-', 'O'),
 ('27', 'CD', '-', 'O'),
 (',', ',', '-', 'O'),
 ('1999', 'CD', '-', 'O'),
 (',', ',', '-', 'O'),
 ('between', 'IN', '-', 'O'),
 ('SILICON', 'NNP', '-', 'I-ORG'),
 ('VALLEY', 'NNP', '-', 'I-ORG'),
 ('BANK', 'NNP', '-', 'I-ORG'),
 ('("', 'NNP', '-', 'O'),
 ('Bank', 'NNP', '-', 'I-ORG'),
 ('"),', ',', '-', 'O'),
 ('a', 'DT', '-', 'O'),
 ('California', 'NNP', '-', 'I-LOC'),
 ('-', ':', '-', 'O'),
 ('chartered', 'VBD', '-', 'O'),
 ('bank', 'NN', '-', 'I-ORG'),
 ('with', 'IN', '-', 'O'),
 ('its', 'PRP$', '-', 'O'),
 ('principal', 'JJ', '-', 'O'),
 ('place', 'NN', '-', 'O'),
 ('of', 'IN', '-', 'O'),
 ('business', 'NN', '-', 'O'),
 ('at', 'IN', '-', 'O'),
 ('3003', 'CD', '-', 'I-LOC'),
 ('Tasman', 'NNP', '-', 'I-LOC'),
 ('Drive', 'NNP', '-', 'I-LOC'),
 (',', ',', '-', 'O'),
 ('Santa', 'NNP',

In [130]:
btc = []
for data in ["a", "b", "e", "f", "g", "h"]:
    btc.extend(load_data("../data/ner_btc/" + data + ".conll", "\t"))
print(len(btc))
btc[1]

9345


[('♥', 'O'),
 ('.', 'O'),
 ('.', 'O'),
 (')', 'O'),
 (')', 'O'),
 ('(', 'O'),
 ('♫', 'O'),
 ('.', 'O'),
 ('(', 'O'),
 ('ړײ', 'O'),
 (')', 'O'),
 ('♫', 'O'),
 ('.', 'O'),
 ('♥', 'O'),
 ('.', 'O'),
 ('«', 'O'),
 ('▓', 'O'),
 ('»', 'O'),
 ('♥', 'O'),
 ('.', 'O'),
 ('♫', 'O'),
 ('.', 'O'),
 ('.', 'O'),
 ('╝', 'O'),
 ('╚', 'O'),
 ('.', 'O'),
 ('.', 'O'),
 ('♫', 'O'),
 ('Happy', 'O'),
 ('New', 'O'),
 ('Year', 'O')]

In [131]:
wiki = load_data("../data/ner_wikigold/" + "wikigold" + ".conll.txt", " ")
print(len(wiki))
wiki[1]

1842


[('This', 'O'),
 ('album', 'O'),
 ('proved', 'O'),
 ('to', 'O'),
 ('be', 'O'),
 ('more', 'O'),
 ('commercial', 'O'),
 ('and', 'O'),
 ('more', 'O'),
 ('techno-based', 'O'),
 ('than', 'O'),
 ('Osc-Dis', 'I-MISC'),
 (',', 'O'),
 ('with', 'O'),
 ('heavily', 'O'),
 ('synthesized', 'O'),
 ('songs', 'O'),
 ('like', 'O'),
 ('Introduction', 'I-MISC'),
 ('010', 'I-MISC'),
 ('and', 'O'),
 ('Come', 'I-MISC'),
 ('.', 'O')]

In [135]:
tokenizer_d = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_d = DistilBertModel.from_pretrained('distilbert-base-uncased')